# PSE検索リスト

## 116項目リスト

# 特定電気用品以外の電気用品（341品目）一覧

# 電気用品安全法 お調べツール

In [18]:
import sys
import pandas as pd
from bs4 import BeautifulSoup
from PySide2.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout,
    QLabel, QLineEdit, QPushButton, QTextEdit, QComboBox
)

class DenanTool(QMainWindow):
    def __init__(self, specified_products, non_specified_products, law_data):
        super().__init__()
        self.specified_products = specified_products
        self.non_specified_products = non_specified_products
        self.law_data = law_data
        self.initUI()

    def initUI(self):
        # Central Widget and Layout
        self.centralWidget = QWidget()
        self.setCentralWidget(self.centralWidget)
        self.mainLayout = QVBoxLayout()
        self.centralWidget.setLayout(self.mainLayout)

        # Category Dropdown
        self.categoryCombo = QComboBox()
        self.categoryCombo.addItems(self.law_data['Category'].unique().tolist())
        self.mainLayout.addWidget(self.categoryCombo)

        # Search Bar
        self.searchLayout = QHBoxLayout()
        self.searchLabel = QLabel("製品検索:")
        self.searchEntry = QLineEdit()
        self.searchButton = QPushButton("検索")
        self.searchLayout.addWidget(self.searchLabel)
        self.searchLayout.addWidget(self.searchEntry)
        self.searchLayout.addWidget(self.searchButton)
        self.mainLayout.addLayout(self.searchLayout)

        # Result Area
        self.resultText = QTextEdit()
        self.resultText.setReadOnly(True)
        self.mainLayout.addWidget(self.resultText)

        # Connect the Search Button to the Search Function
        self.searchButton.clicked.connect(self.searchFunction)

    def searchFunction(self):
        # Implement the search logic here
        search_query = self.searchEntry.text().lower()
        category = self.categoryCombo.currentText()
        results = self.specified_products[self.specified_products['Category'] == category]
        results = results[results['Product'].str.contains(search_query, case=False, na=False)]

        # Display the results
        self.resultText.setPlainText(results.to_string(index=False))

def read_specified_products(filepath):
    # Read the Excel file for specified electrical products
    return pd.read_excel(filepath)

def read_non_specified_products(filepath):
    # Read the Excel file for non-specified electrical products
    return pd.read_excel(filepath)

def read_law_data(filepath):
    # Read the HTML file for law categories
    with open(filepath, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
    categories = soup.find_all('h2')
    return pd.DataFrame({'Category': [cat.get_text() for cat in categories]})

# Paths to the files - replace with the actual paths
specified_products_path = 'specified_electrical_products.xls'
non_specified_products_path = 'non-specified_electrical_products.xls'
law_data_path = '電気用品安全法施行令.htm'

# Read the data from the files
specified_products = read_specified_products(specified_products_path)
non_specified_products = read_non_specified_products(non_specified_products_path)
law_data = read_law_data(law_data_path)

# Create an instance of QApplication
app = QApplication(sys.argv)

# Create an instance of the application's main window
mainWindow = DenanTool(specified_products, non_specified_products, law_data)
mainWindow.show()

# Enter the application's main loop
sys.exit(app.exec_())


ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [1]:
import sys
import pandas as pd
import webbrowser
from PySide2.QtWidgets import QApplication, QWidget, QVBoxLayout, QLineEdit, QTextEdit, QPushButton
from PySide2.QtCore import Slot

class PSEChecker(QWidget):
    def __init__(self):
        super().__init__()
        self.non_specified_df = None
        self.specified_df = None
        self.init_ui()

    def init_ui(self):
        self.setWindowTitle('電気用品安全法 お調べツール')
        layout = QVBoxLayout()

        self.search_box = QLineEdit(self)
        self.search_box.setPlaceholderText('製品名を入力してください')
        layout.addWidget(self.search_box)

        self.search_button = QPushButton('検索', self)
        self.search_button.clicked.connect(self.on_search_button_clicked)
        layout.addWidget(self.search_button)

        self.result_area = QTextEdit(self)
        self.result_area.setReadOnly(True)
        layout.addWidget(self.result_area)

        self.setLayout(layout)

    def load_excel_data(self):
        try:
            self.non_specified_df = pd.read_excel('non-specified_electrical_products.xlsx')
            self.specified_df = pd.read_excel('specified_electrical_products.xlsx')
        except Exception as e:
            self.result_area.append(f"Excelファイルの読み込みに失敗しました: {e}")

    def search_local_data(self, product_name):
        # 非特定電気用品リストから検索
        non_specified_results = self.non_specified_df[self.non_specified_df['Product'].str.contains(product_name, na=False)]
        # 特定電気用品リストから検索
        specified_results = self.specified_df[self.specified_df['Product'].str.contains(product_name, na=False)]

        return non_specified_results, specified_results

    @Slot()
    def on_search_button_clicked(self):
        product_name = self.search_box.text()
        non_specified_results, specified_results = self.search_local_data(product_name)

        # 結果を表示する
        if not non_specified_results.empty:
            self.result_area.append("非特定電気用品リストで見つかりました:\n" + non_specified_results.to_string(index=False))
        if not specified_results.empty:
            self.result_area.append("特定電気用品リストで見つかりました:\n" + specified_results.to_string(index=False))
        if non_specified_results.empty and specified_results.empty:
            search_url = f"https://www.google.com/search?q={product_name.replace(' ', '+')}"
            webbrowser.open(search_url)
            self.result_area.append(f"ローカルには見つかりませんでした。Webで検索: {search_url}")
            search_url = f"https://www.google.com/search?q={product_name.replace(' ', '+')}"
            webbrowser.open(search_url)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    if not QApplication.instance():
        app = QApplication(sys.argv)
    checker = PSEChecker()
    checker.load_excel_data()
    checker.show()
    sys.exit(app.exec_())


SystemExit: 0

C:\Users\Manyan\anaconda3\envs\osac\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
